In [1]:
import pandas as pd
import numpy as np

In [2]:
# nature dataset

In [4]:
nat = pd.read_excel("Nature Subjects Ontology-March-2020.xlsx", usecols=['Code', 'Subject name', 'Alternative label(s)'])[['Code', 'Subject name', 'Alternative label(s)']]
nat.head()

,Code,Subject name,Alternative label(s)
0,2221,3-D reconstruction,Tomographic reconstruction
1,2665,Abiotic,NaN
2,1275,Acetylation,Acetylations
3,1275,Acetylation,Acetylations
4,1275,Acetylation,Acetylations


In [6]:
nat = nat.drop_duplicates()

In [7]:
nat.shape[0]

2851

In [8]:
nat.columns = ['nature_code', 'nature_subject_name', 'nature_synonyms']

In [9]:
nat['nature_subject_name'] = nat['nature_subject_name'].str.strip().str.lower()
nat['nature_synonyms'] = nat['nature_synonyms'].str.strip().str.lower()

C:\Users\cbq2413\AppData\Local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\cbq2413\AppData\Local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
# snt dataset

In [11]:
snt = pd.read_excel("ontology_2020-05-29_09-00-35.xlsx", usecols=['Subject ID', 'subject-name', 'Synonyms', 'Level'])[['Subject ID', 'subject-name', 'Synonyms', 'Level']]
snt.head()

,Subject ID,subject-name,Synonyms,Level
0,2874,Life Sciences,NaN,1
1,2873,Biological Sciences,"Biologic Sciences, Biological Science, Life Sc...",2
2,2935,Neuroscience,Neurosciences,3
3,2872,Agriculture,"Agricultural Science, Farming",3
4,2921,Plant Science,"Plant biology, Plant Sciences",3


In [12]:
snt = snt.drop(columns=['Level'])

In [13]:
snt = snt.drop_duplicates()

In [14]:
snt.shape[0]

4794

In [15]:
snt.columns = ['snt_id', 'snt_subject_name', 'snt_synonyms']

In [16]:
snt['snt_subject_name'] = snt['snt_subject_name'].str.strip().str.lower()
snt['snt_synonyms'] = snt['snt_synonyms'].str.strip().str.lower()

# Mapping

In [17]:
#direct matching

In [33]:
df = pd.merge(nat, snt, how='left', left_on='nature_subject_name', right_on='snt_subject_name')

In [34]:
df.loc[~df['snt_id'].isna(),'match_type'] = 'direct'

In [35]:
df.head(10)

,nature_code,nature_subject_name,nature_synonyms,snt_id,snt_subject_name,snt_synonyms,match_type
0,2221,3-d reconstruction,tomographic reconstruction,NaN,NaN,NaN,NaN
1,2665,abiotic,NaN,5497.0,abiotic,NaN,direct
2,1275,acetylation,acetylations,7142.0,acetylation,acetylations,direct
3,1239,acetyltransferases,transacetylase,4514.0,acetyltransferases,transacetylase,direct
4,1475,achalasia,"achalasias,cardiospasm,cardiospasms,esophageal...",3484.0,achalasia,"cardiospasm, esophageal achalasias, achalasias...",direct
5,1,"acid, base, fluid, electrolyte disorders",NaN,5633.0,"acid, base, fluid, electrolyte disorders",NaN,direct
6,4035,acne vulgaris,NaN,6336.0,acne vulgaris,NaN,direct
7,3927,acoustics,NaN,5882.0,acoustics,NaN,direct
8,1276,actin,"actins,f actin,f-actin,g actin,g-actin,isoacti...",6503.0,actin,"alpha-isoactin, alpha actin, g-actin, isoactin...",direct
9,1277,action potential generation,"action potential,action potentials,nerve impul...",5477.0,action potential generation,"nerve impulse, action potential, spike potenti...",direct


In [36]:
df[df['match_type'] == 'direct'].shape[0], df.shape[0]

(2375, 2852)

In [37]:
#label-synonym matching

In [46]:
for i in df[df['match_type'] != 'direct'].index:
    A = [x.strip() for x in df.at[i, 'nature_subject_name'].split(',')]
    A_plus = []
    if str(df.at[i, 'nature_synonyms']) != 'nan':
        A_plus = A + [x.strip() for x in df.at[i, 'nature_synonyms'].split(',')]
    for j in snt.index:
        match = False
        B = [x.strip() for x in snt.at[j, 'snt_subject_name'].split(',')]
        B_plus = []
        if str(snt.at[j, 'snt_synonyms']) != 'nan':
            B_plus = B + [x.strip() for x in snt.at[j, 'snt_synonyms'].split(',')]
        if bool(list(set(A) & set(B_plus))) | bool(list(set(A_plus) & set(B))) | bool(list(set(A) & set(B))):
            df.loc[i, 'snt_id'] = snt.at[j, 'snt_id']
            df.loc[i, 'snt_subject_name'] = snt.at[j, 'snt_subject_name']
            df.loc[i, 'snt_synonyms'] = snt.at[j, 'snt_synonyms']
            df.loc[i, 'match_type'] = 'label-synonym'
            break

In [47]:
#synonym-synonym matching

In [48]:
for i in df[~df['match_type'].isin(['direct', 'label-synonym'])].index:
    if str(df.at[i, 'nature_synonyms']) != 'nan':
        A = [x.strip() for x in df.at[i, 'nature_synonyms'].split(',')]
        for j in snt.index:
            if str(snt.at[j, 'snt_synonyms']) != 'nan':
                B = [x.strip() for x in snt.at[j, 'snt_synonyms'].split(',')]
                if list(set(A) & set(B)):
                    df.loc[i, 'snt_id'] = snt.at[j, 'snt_id']
                    df.loc[i, 'snt_subject_name'] = snt.at[j, 'snt_subject_name']
                    df.loc[i, 'snt_synonyms'] = snt.at[j, 'snt_synonyms']
                    df.loc[i, 'match_type'] = 'synonym-synonym'
                    break

In [49]:
df[df['match_type'] == 'label-synonym'].shape[0]

107

In [50]:
df[df['match_type'] == 'synonym-synonym'].shape[0]

42

In [51]:
df[df['match_type'].isna()].shape[0]

328

In [57]:
df['snt_id'] = df['snt_id'].astype('Int64')

In [61]:
# Save

In [60]:
multi_assign_snt = df[(df.duplicated(subset='snt_id', keep=False)) & ~df['snt_id'].isna()].sort_values(by='snt_id')
multi_assign_snt.to_excel('multi_assign_snt.xlsx', index=False)

In [62]:
df.to_excel('nature_subjects_to_snt.xlsx', index=False)

In [63]:
import datetime

In [67]:
datetime.datetime.strftime(datetime.date.today(), format="As of %Y-%m-%d")

'As of 2020-09-01'

In [68]:
help(df.to_excel)

Help on method to_excel in module pandas.core.generic:

to_excel(excel_writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True, freeze_panes=None) method of pandas.core.frame.DataFrame instance
    Write object to an Excel sheet.
    
    To write a single object to an Excel .xlsx file it is only necessary to
    specify a target file name. To write to multiple sheets it is necessary to
    create an `ExcelWriter` object with a target file name, and specify a sheet
    in the file to write to.
    
    Multiple sheets may be written to by specifying unique `sheet_name`.
    With all data written to the file it is necessary to save the changes.
    Note that creating an `ExcelWriter` object with a file name that already
    exists will result in the contents of the existing file being erased.
    
    Parameters
    ----------
    